# Hyperparameter Optimization for Neural Apoptosis

**Goal:** Find optimal hyperparameters for neuron-level apoptosis strategies

**Strategies tested:**
1. Standard Neuron Apoptosis (mutation-based)
2. Functional Preservation Apoptosis (output matching)
3. Hybrid Growth and Death (constant capacity evolution)

**Status:** See `EXPERIMENT_LOG.md` for previous results

## Setup

In [1]:
import sys
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path

# Add src to path for imports
sys.path.insert(0, str(Path.cwd() / 'src'))

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

PyTorch version: 2.9.0
CUDA available: False
MPS available: True


In [2]:
# Device selection (prioritize MPS for M2 Max)
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA")
else:
    device = torch.device('cpu')
    print("Using CPU")

print(f"Device: {device}")

Using MPS (Apple Silicon GPU)
Device: mps


## Load Data

In [3]:
# Simple character-level tokenizer
class CharTokenizer:
    def __init__(self, text):
        chars = sorted(list(set(text)))
        self.vocab_size = len(chars)
        self.stoi = {ch: i for i, ch in enumerate(chars)}
        self.itos = {i: ch for i, ch in enumerate(chars)}
    
    def encode(self, text):
        return [self.stoi[c] for c in text]
    
    def decode(self, tokens):
        return ''.join([self.itos[i] for i in tokens])

# Dataset
class CharDataset(torch.utils.data.Dataset):
    def __init__(self, text, tokenizer, seq_len=128):
        self.data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.data) - self.seq_len
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_len]
        y = self.data[idx + 1:idx + self.seq_len + 1]
        return x, y

# Load Shakespeare
import urllib.request
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
print("Downloading Shakespeare dataset...")
with urllib.request.urlopen(url) as response:
    shakespeare_text = response.read().decode('utf-8')

print(f"Dataset size: {len(shakespeare_text):,} characters")
print(f"Preview: {shakespeare_text[:200]}")

Dataset size: 1,115,394 characters
Preview: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
# Create tokenizer and dataset
tokenizer = CharTokenizer(shakespeare_text)
shakespeare_dataset = CharDataset(shakespeare_text, tokenizer, seq_len=128)

print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Dataset length: {len(shakespeare_dataset):,} sequences")

Vocabulary size: 65
Dataset length: 1,115,266 sequences


## Model Definition

In [5]:
# Simple Transformer
class ApoptoticTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, n_layers=6, max_seq_len=128, enable_apoptosis=False):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)
        
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads) for _ in range(n_layers)
        ])
        
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        B, T = x.shape
        
        tok_emb = self.embedding(x)
        pos_emb = self.pos_embedding(torch.arange(T, device=x.device))
        x = tok_emb + pos_emb
        
        for block in self.blocks:
            x = block(x)
        
        x = self.ln_f(x)
        logits = self.head(x)
        return logits
    
    def get_num_params(self):
        return sum(p.numel() for p in self.parameters())


class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, n_heads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
    
    def forward(self, x):
        # Attention
        attn_out, _ = self.attention(x, x, x, need_weights=False)
        x = x + attn_out
        x = self.ln1(x)
        
        # FFN
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.ln2(x)
        
        return x

# Test model creation
test_model = ApoptoticTransformer(
    vocab_size=tokenizer.vocab_size,
    d_model=128,
    n_heads=4,
    n_layers=6,
    max_seq_len=128
).to(device)

print(f"Model parameters: {test_model.get_num_params():,}")

Model parameters: 1,222,977


## Import Apoptosis Strategies

In [6]:
# Import from src/ package
from neuron_apoptosis_fixed import NeuronApoptosisManager
from smooth_apoptosis import FunctionalPreservationApoptosis
from growth_only_strategy import HybridGrowthAndDeath
from hyperparameter_sweep import HyperparameterSweep, QuickSweep

print("✓ All apoptosis strategies loaded!")

✓ Neuron-level apoptosis system loaded!
✓ Neuron-level apoptosis system loaded!
✓ Smooth apoptosis strategies loaded!

Available strategies:
  1. GradualFadeApoptosis - Fade neurons over 50 steps
  2. ContinuousTurnoverApoptosis - 2% every 100 steps
  3. DistillationApoptosis - Train new neurons first
  4. FunctionalPreservationApoptosis - Preserve function

Run compare_strategies() to test all!
✓ Growth-only strategies loaded!

Available:
  1. GrowthOnlyManager - Add neurons, never remove
  2. HybridGrowthAndDeath - Constant turnover (birth = death)
✓ Genetic crossover strategy loaded!

Usage:
  # Create crossover manager
  crossover_mgr = CrossoverApoptosis(
      model=model,
      target_layers=[f'blocks.{i}.ffn.0' for i in range(6)],
      prune_rate=0.10,
      apoptosis_interval=500,
      crossover_mode='uniform',  # or 'fitness_weighted', 'random'
      mutation_strength=0.1
  )

  # Use in training loop
  crossover_mgr.step()

Crossover modes:
  - uniform: 50/50 blend of pare

In [7]:
# Reload modules to pick up any code changes
import importlib
import sys

if 'hyperparameter_sweep' in sys.modules:
    importlib.reload(sys.modules['hyperparameter_sweep'])

from hyperparameter_sweep import HyperparameterSweep, QuickSweep

print("✓ Modules reloaded!")

✓ Hyperparameter sweep system loaded!

Usage:
  # Full sweep (may take hours)
  sweep = HyperparameterSweep(
      model_class=ApoptoticTransformer,
      train_dataset=shakespeare_dataset,
      val_dataset=shakespeare_dataset,
      device=device,
      num_steps=2000
  )
  sweep.run_sweep()

  # Quick sweep (10-15 configs, ~1 hour)
  quick_sweep = QuickSweep(...same args...)
  quick_sweep.run_sweep()
✓ Modules reloaded!


## Baseline Training (No Apoptosis)

In [8]:
# Train baseline model
baseline_model = ApoptoticTransformer(
    vocab_size=tokenizer.vocab_size,
    d_model=128,
    n_heads=4,
    n_layers=6,
    max_seq_len=128
).to(device)

# Training
train_loader = torch.utils.data.DataLoader(
    shakespeare_dataset, batch_size=64, shuffle=True, num_workers=0
)

optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

baseline_losses = []
train_iter = iter(train_loader)

from tqdm import tqdm
print("Training baseline model (2000 steps)...")
pbar = tqdm(total=2000)

for step in range(2000):
    try:
        x, y = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        x, y = next(train_iter)
    
    x, y = x.to(device), y.to(device)
    
    baseline_model.train()
    logits = baseline_model(x)
    loss = criterion(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(baseline_model.parameters(), max_norm=1.0)
    optimizer.step()
    
    baseline_losses.append(loss.item())
    
    if step % 100 == 0:
        pbar.set_postfix({'loss': f'{loss.item():.3f}'})
    pbar.update(1)

pbar.close()

baseline_final_loss = np.mean(baseline_losses[-100:])
print(f"\n✓ Baseline training complete!")
print(f"Final loss: {baseline_final_loss:.4f}")

Training baseline model (2000 steps)...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:01<00:00, 32.68it/s, loss=0.016]


✓ Baseline training complete!
Final loss: 0.0189


## Hyperparameter Sweep

Test multiple configurations to find optimal parameters.

In [9]:
# Run quick sweep (13 configs)

sweep = QuickSweep(
    model_class=ApoptoticTransformer,
    train_dataset=shakespeare_dataset,
    val_dataset=shakespeare_dataset,
    device=device,
    tokenizer=tokenizer,
    num_steps=2000,
    eval_interval=100
)

print("Starting hyperparameter sweep...")
sweep.run_sweep()

Starting hyperparameter sweep...
Quick sweep - testing 13 configurations

HYPERPARAMETER SWEEP
Total experiments: 13
Steps per experiment: 2000
Estimated time: 65 minutes (~5 min per config)



Running experiments:   0%|                                                                                                                                    | 0/13 [00:00<?, ?it/s]


[1/13] Testing: standard

[Neuron Apoptosis @ step 500] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0001, 0.0008]

[Neuron Apoptosis @ step 500] blocks.1.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0008]

[Neuron Apoptosis @ step 500] blocks.2.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.3.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0008]

[Neuron Apoptosis @ step 500] blocks.4.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.5.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 1000] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0008]

[Neuron Apoptosis @ step 1000] blocks.1.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0013]

[Neuron Apoptosis @ step 1000] blocks.2.ffn.0
  Pruning 51 

Running experiments:   8%|█████████▌                                                                                                                  | 1/13 [01:11<14:22, 71.86s/it]


[Neuron Apoptosis @ step 2000] blocks.5.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0015]
  Final loss: 0.0207
  Events: 24

[2/13] Testing: standard

[Neuron Apoptosis @ step 500] blocks.0.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0001, 0.0007]

[Neuron Apoptosis @ step 500] blocks.1.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0001, 0.0009]

[Neuron Apoptosis @ step 500] blocks.2.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 500] blocks.3.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.4.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 500] blocks.5.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0011]

[Neuron Apoptosis @ step 1000] blocks.0.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0010]

[Neuron Apoptosis @ step 

...


[Neuron Apoptosis @ step 2000] blocks.5.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0015]
  Final loss: 0.0209
  Events: 30

[12/13] Testing: hybrid

[Neuron Apoptosis @ step 500] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0001, 0.0009]

[Neuron Apoptosis @ step 500] blocks.1.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0001, 0.0007]

[Neuron Apoptosis @ step 500] blocks.2.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0008]

[Neuron Apoptosis @ step 500] blocks.3.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0009]

[Neuron Apoptosis @ step 500] blocks.4.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.5.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0010]

[Neuron Apoptosis @ step 1000] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0011]

[Neuron Apoptosis @ step 1

Running experiments:  92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌         | 12/13 [28:00<01:24, 84.97s/it]

  Final loss: 0.0209
  Events: 24

[13/13] Testing: hybrid

[Neuron Apoptosis @ step 600] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0001, 0.0007]

[Neuron Apoptosis @ step 600] blocks.1.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 600] blocks.2.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 600] blocks.3.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0005]

[Neuron Apoptosis @ step 600] blocks.4.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 600] blocks.5.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 1200] blocks.0.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0017]

[Neuron Apoptosis @ step 1200] blocks.1.ffn.0
  Pruning 51 neurons (bottom 10%)
  Fitness range: [0.0000, 0.0016]

[Neuron Apoptosis @ step 1

Running experiments: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [29:11<00:00, 134.77s/it]

  Final loss: 0.0196
  Events: 18

Results saved to: sweep_results_20251120_185323.json

TOP CONFIGURATIONS

🏆 TOP 10 BY FINAL LOSS:

1. FUNCTIONAL
   Loss: 0.0190
   Variance: 0.000025
   Events: 24
   Config: {'strategy': 'functional', 'prune_rate': 0.15, 'interval': 500, 'mutation_strength': 0.3}

2. HYBRID
   Loss: 0.0193
   Variance: 0.000265
   Events: 24
   Config: {'strategy': 'hybrid', 'turnover_rate': 0.08, 'interval': 500, 'mutation_strength': 0.3}

3. HYBRID
   Loss: 0.0196
   Variance: 0.001576
   Events: 18
   Config: {'strategy': 'hybrid', 'turnover_rate': 0.1, 'interval': 600, 'mutation_strength': 0.3}

4. STANDARD
   Loss: 0.0200
   Variance: 0.004482
   Events: 24
   Config: {'strategy': 'standard', 'prune_rate': 0.15, 'interval': 500, 'mutation_strength': 0.3, 'fitness_metric': 'grad_activation', 'regrowth_strategy': 'mutation'}

5. FUNCTIONAL
   Loss: 0.0203
   Variance: 0.000003
   Events: 24
   Config: {'strategy': 'functional', 'prune_rate': 0.1, 'interval': 500,

## Results Analysis

In [12]:
# Print top results
print("\n" + "="*70)
print("TOP CONFIGURATIONS")
print("="*70)

valid_results = [r for r in sweep.results if 'error' not in r]
sorted_results = sorted(valid_results, key=lambda x: x['final_train_loss'])

print(f"\nBaseline: {baseline_final_loss:.4f}")
print("\nTop 5:")

for i, result in enumerate(sorted_results[:5], 1):
    config = result['config']
    loss = result['final_train_loss']
    variance = result['train_loss_variance']
    
    diff = loss - baseline_final_loss
    pct = (diff / baseline_final_loss) * 100
    
    print(f"\n{i}. {config['strategy'].upper()}")
    print(f"   Loss: {loss:.4f} ({diff:+.4f}, {pct:+.1f}% from baseline)")
    print(f"   Variance: {variance:.6f}")
    print(f"   Events: {result['num_events']}")
    print(f"   Config: {config}")


TOP CONFIGURATIONS

Baseline: 0.0189

Top 5:

1. FUNCTIONAL
   Loss: 0.0190 (+0.0001, +0.3% from baseline)
   Variance: 0.000025
   Events: 24
   Config: {'strategy': 'functional', 'prune_rate': 0.15, 'interval': 500, 'mutation_strength': 0.3}

2. HYBRID
   Loss: 0.0193 (+0.0004, +1.9% from baseline)
   Variance: 0.000265
   Events: 24
   Config: {'strategy': 'hybrid', 'turnover_rate': 0.08, 'interval': 500, 'mutation_strength': 0.3}

3. HYBRID
   Loss: 0.0196 (+0.0007, +3.8% from baseline)
   Variance: 0.001576
   Events: 18
   Config: {'strategy': 'hybrid', 'turnover_rate': 0.1, 'interval': 600, 'mutation_strength': 0.3}

4. STANDARD
   Loss: 0.0200 (+0.0010, +5.5% from baseline)
   Variance: 0.004482
   Events: 24
   Config: {'strategy': 'standard', 'prune_rate': 0.15, 'interval': 500, 'mutation_strength': 0.3, 'fitness_metric': 'grad_activation', 'regrowth_strategy': 'mutation'}

5. FUNCTIONAL
   Loss: 0.0203 (+0.0014, +7.4% from baseline)
   Variance: 0.000003
   Events: 24
   Co

## Validate Winner (5K Steps)

In [ ]:
# Get best config from sweep
best_result = sorted_results[0]
best_config = best_result['config']

print(f"Validating best config: {best_config['strategy']}")
print(f"Expected loss: ~{best_result['final_train_loss']:.4f}")

# Create model with best config
validation_model = ApoptoticTransformer(
    vocab_size=tokenizer.vocab_size,
    d_model=128,
    n_heads=4,
    n_layers=6,
    max_seq_len=128
).to(device)

# Create manager based on strategy
target_layers = [f'blocks.{i}.ffn.0' for i in range(6)]

if best_config['strategy'] == 'standard':
    manager = NeuronApoptosisManager(
        model=validation_model,
        target_layers=target_layers,
        **{k: v for k, v in best_config.items() if k != 'strategy'}
    )
elif best_config['strategy'] == 'functional':
    manager = FunctionalPreservationApoptosis(
        model=validation_model,
        target_layers=target_layers,
        **{k: v for k, v in best_config.items() if k != 'strategy' and k != 'interval'}
    )
elif best_config['strategy'] == 'hybrid':
    manager = HybridGrowthAndDeath(
        model=validation_model,
        target_layers=target_layers,
        **{k: v for k, v in best_config.items() if k != 'strategy'}
    )

# Train for 5K steps
optimizer = torch.optim.AdamW(validation_model.parameters(), lr=3e-4)
val_losses = []
train_iter = iter(train_loader)

print("\nTraining for 5000 steps...")
pbar = tqdm(total=5000)

for step in range(5000):
    try:
        x, y = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        x, y = next(train_iter)
    
    x, y = x.to(device), y.to(device)
    
    validation_model.train()
    logits = validation_model(x)
    loss = criterion(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(validation_model.parameters(), max_norm=1.0)
    optimizer.step()
    
    manager.step()
    
    val_losses.append(loss.item())
    
    if step % 100 == 0:
        pbar.set_postfix({'loss': f'{loss.item():.3f}'})
    pbar.update(1)

pbar.close()

validation_final_loss = np.mean(val_losses[-100:])
print(f"\n✓ Validation complete!")
print(f"\nFinal Results:")
print(f"  Baseline:   {baseline_final_loss:.4f}")
print(f"  Best config: {validation_final_loss:.4f}")
print(f"  Difference: {validation_final_loss - baseline_final_loss:+.4f}")

Validating best config: functional
Expected loss: ~0.0190

Training for 5000 steps...


 10%|████████████▊                                                                                                                    | 499/5000 [00:15<02:15, 33.24it/s, loss=0.071]


[Neuron Apoptosis @ step 500] blocks.0.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0001, 0.0006]


 10%|████████████▊                                                                                                                    | 499/5000 [00:25<02:15, 33.24it/s, loss=0.071]


[Neuron Apoptosis @ step 500] blocks.1.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0001, 0.0006]

[Neuron Apoptosis @ step 500] blocks.2.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.3.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 500] blocks.4.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 500] blocks.5.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0010]


...


[Neuron Apoptosis @ step 1500] blocks.1.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0010]

[Neuron Apoptosis @ step 1500] blocks.2.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 1500] blocks.3.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0007]

[Neuron Apoptosis @ step 1500] blocks.4.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0006]

[Neuron Apoptosis @ step 1500] blocks.5.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0014]


 40%|███████████████████████████████████████████████████                                                                             | 1997/5000 [07:30<01:31, 32.79it/s, loss=0.019]


[Neuron Apoptosis @ step 2000] blocks.0.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0011]


 40%|███████████████████████████████████████████████████▏                                                                            | 1999/5000 [07:46<01:31, 32.79it/s, loss=0.019]


[Neuron Apoptosis @ step 2000] blocks.1.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0008]

[Neuron Apoptosis @ step 2000] blocks.2.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0010]

[Neuron Apoptosis @ step 2000] blocks.3.ffn.0
  Pruning 76 neurons (bottom 15%)
  Fitness range: [0.0000, 0.0005]


## Save Results

In [ ]:
import json
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save sweep results
with open(f'results/sweep_{timestamp}.json', 'w') as f:
    json.dump(sweep.results, f, indent=2)

# Save validation results
validation_results = {
    'config': best_config,
    'baseline_loss': baseline_final_loss,
    'validation_loss': validation_final_loss,
    'num_events': len(manager.apoptosis_events) if hasattr(manager, 'apoptosis_events') else 0,
    'timestamp': timestamp
}

with open(f'results/validation_{timestamp}.json', 'w') as f:
    json.dump(validation_results, f, indent=2)

print(f"✓ Results saved to results/")

## Next Steps

1. Update `EXPERIMENT_LOG.md` with results
2. Try Taguchi optimization for faster search
3. Test domain shift (Shakespeare → Wikipedia)
4. Visualize neuron lineages